In [1]:
import os
import torchvision
import sys
import dotenv
sys.path.append("../..")
from src.transforms import ImageResizer,make_patches
from tqdm.notebook import tqdm

/home/abdelnour/anaconda3/envs/ml/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/abdelnour/anaconda3/envs/ml/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [5]:
ROI_DIR = dotenv.get_key(dotenv.find_dotenv(), "DATA_DIR")
PATCHED_ROI_DIR = '/home/abdelnour/Documents/4eme_anne/S2/projet/data/patched'

print(ROI_DIR)
print(PATCHED_ROI_DIR)

/home/abdelnour/Documents/4eme_anne/S2/projet/data/roi-dataset/BRACS_RoI/latest_version
/home/abdelnour/Documents/4eme_anne/S2/projet/data/patched


In [7]:
resizer = ImageResizer()
to_tensor = torchvision.transforms.ToTensor()
to_pil = torchvision.transforms.ToPILImage()

In [69]:
class RoiIDataset(torchvision.datasets.ImageFolder):
    
    def __getitem__(self, index: int):
        img, label = super().__getitem__(index)
        path = self.imgs[index][0]
        return path, img, label


In [79]:
splits = os.listdir(ROI_DIR)

for split in splits:

    print(f"split = {split}")

    # create a new dataset
    dataset = RoiIDataset(root=os.path.join(ROI_DIR, split))

    # create a folder for this particular split
    os.mkdir(os.path.join(PATCHED_ROI_DIR, split))

    for class_ in dataset.classes:
        os.mkdir(os.path.join(PATCHED_ROI_DIR, split, class_))
        
    for path,img,label in tqdm(dataset): 

        class_ = dataset.classes[label]       
        
        # resize the image
        resized_img = resizer(img)

        # convert the PIL image to tensor
        tensor_img = to_tensor(resized_img)

        # get a list of patches
        patches = make_patches(tensor_img, 224, 224)

        basename = os.path.basename(path)
        name,extention = os.path.splitext(basename)

        # save the patches to the disk
        for i,patch in enumerate(patches):
            pil_patch = to_pil(patch)
            patch_path = os.path.join(PATCHED_ROI_DIR, split, class_, f'{name}_{i}.{extention}')
            pil_patch.save(patch_path)
            

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/570 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [78]:
!rm -r $PATCHED_ROI_DIR/*